In [ ]:
# Important links

# https://github.com/crowdAI/coco/tree/master/PythonAPI/pycocotools
# https://github.com/crowdAI/mapping-challenge-starter-kit/blob/master/Dataset%20Utils.ipynb
# https://arxiv.org/pdf/1703.06870.pdf

In [1]:
%matplotlib inline
from pycocotools.coco import COCO
from pycocotools import mask as cocomask
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import os
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [6]:
project_dir = "/home/webwerks/patricia/my-projects/github/Segmentation"
data_directory = "data/"
annotation_file_template = "{}/{}/annotation{}.json"

TRAIN_IMAGES_DIRECTORY = "data/train/images"
TRAIN_ANNOTATIONS_PATH = "data/train/annotation.json"
TRAIN_ANNOTATIONS_SMALL_PATH = "data/train/annotation-small.json"

VAL_IMAGES_DIRECTORY = "data/val/images"
VAL_ANNOTATIONS_PATH = "data/val/annotation.json"
VAL_ANNOTATIONS_SMALL_PATH = "data/val/annotation-small.json"

In [8]:
train_coco = COCO(os.path.join(project_dir, TRAIN_ANNOTATIONS_PATH ))

loading annotations into memory...
Done (t=34.45s)
creating index...
index created!


Help on method annToMask in module pycocotools.coco:

annToMask(ann) method of pycocotools.coco.COCO instance
    Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
    :return: binary mask (numpy 2D array)

